In [17]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%autoreload 2

In [19]:
%matplotlib inline
import sys
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *

LANDSCAPE_TYPES ={"RNA": [0, 1], 'TF':['SIX6_REF_R1']} # for testing 
LANDSCAPE_TYPES_RNA = {"RNA" : [0,1,12,20,25,31], "TF": []}
LANDSCAPE_TYPES_TF = {"RNA": [], 
                      "TF": ['POU3F4_REF_R1','PAX3_G48R_R1','SIX6_REF_R1', 'VAX2_REF_R1', 'VSX1_REF_R1']}
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
DNA_alphabet = "TCGA"
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


B1L14RNA1 loaded


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## DQN Explorer

In [8]:
from explorers.dqn_explorer import DQN_Explorer 

num_of_rounds = 5

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(train_epochs=10)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.14554841726711917
round: 1, cost: 1, evals: 0, top: 0.14554841726711917
round: 2, cost: 1, evals: 0, top: 0.14554841726711917
round: 3, cost: 1, evals: 0, top: 0.14554841726711917
round: 4, cost: 1, evals: 0, top: 0.14554841726711917


In [15]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object = next(landscape_generator)
wt = landscape_object['starting_seqs']['TF0']
landscape_oracle = landscape_object['landscape_oracle']

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=DNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(alphabet=DNA_alphabet, train_epochs=10, virtual_screen=20)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

1 TF landscapes loaded.


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.6781429101554447
round: 1, cost: 1, evals: 0, top: 0.6781429101554447
round: 2, cost: 1, evals: 0, top: 0.6781429101554447
round: 3, cost: 1, evals: 0, top: 0.6781429101554447
round: 4, cost: 1, evals: 0, top: 0.6781429101554447


In [20]:
from explorers.dqn_explorer import DQN_Explorer 

dqn_explorer_prod = DQN_Explorer(train_epochs=10)
dqn_explorer_prod.debug = False

In [21]:
# run all evaluations 
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=5)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=5)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=5)

loading landscapes RNA: [0, 1, 12, 20, 25, 31], TF:[]
B1L14RNA1 loaded
B1L50RNA1 loaded
B2L14RNA1+2 loaded
B2L50RNA1+4 loaded
B2L100RNA1+3 loaded
C20B2L100RNA1+3 loaded
0 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


start seq startRNAL14_1
start seq startRNAL14_2
Running on B1L50RNA1
start seq startRNAL50_0
start seq startRNAL50_1
start seq startRNAL50_2
Running on B2L14RNA1+2
start seq startRNAL14_0
start seq startRNAL14_1
start seq startRNAL14_2
Running on B2L50RNA1+4
start seq startRNAL50_0
start seq startRNAL50_1
start seq startRNAL50_2
Running on B2L100RNA1+3
start seq startRNAL100_0
start seq startRNAL100_1
start seq startRNAL100_2
Running on C20B2L100RNA1+3
start seq startRNAL100_0
start seq startRNAL100_1
start seq startRNAL100_2
loading landscapes RNA: [], TF:['POU3F4_REF_R1', 'PAX3_G48R_R1', 'SIX6_REF_R1', 'VAX2_REF_R1', 'VSX1_REF_R1']
5 TF landscapes loaded.
loading complete
Running on POU3F4_REF_R1
start seq TF0
start seq TF1
start seq TF2
start seq TF3
start seq TF4
Running on PAX3_G48R_R1
start seq TF0
start seq TF1
start seq TF2
start seq TF3
start seq TF4
Running on SIX6_REF_R1
start seq TF0
start seq TF1
start seq TF2
start seq TF3
start seq TF4
Running on VAX2_REF_R1
start seq TF

POU3F4_REF_R1,1000,100,108.33840560913086

start seq TF2
POU3F4_REF_R1,100,1,0.18558120727539062

POU3F4_REF_R1,100,10,1.1254701614379883

POU3F4_REF_R1,100,100,10.183694839477539

POU3F4_REF_R1,1000,1,1.990445852279663

POU3F4_REF_R1,1000,10,10.798193216323853

POU3F4_REF_R1,1000,100,108.69755554199219

start seq TF3
POU3F4_REF_R1,100,1,0.15920329093933105

POU3F4_REF_R1,100,10,1.101102590560913

POU3F4_REF_R1,100,100,9.955430746078491

POU3F4_REF_R1,1000,1,1.496490240097046

POU3F4_REF_R1,1000,10,10.909098148345947

POU3F4_REF_R1,1000,100,99.83661842346191

start seq TF4
POU3F4_REF_R1,100,1,0.2188739776611328

POU3F4_REF_R1,100,10,1.1076173782348633

POU3F4_REF_R1,100,100,10.893426895141602

POU3F4_REF_R1,1000,1,1.9809165000915527

POU3F4_REF_R1,1000,10,10.936813354492188

POU3F4_REF_R1,1000,100,100.02221059799194

Running on PAX3_G48R_R1
start seq TF0
PAX3_G48R_R1,100,1,0.22397160530090332

PAX3_G48R_R1,100,10,1.1039106845855713

PAX3_G48R_R1,100,100,10.950556516647339

PAX3_G48R_R1

In [ ]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.consistency_robustness_independence, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.consistency_robustness_independence, num_starts=5)

loading landscapes RNA: [0, 1, 12, 20, 25, 31], TF:[]
B1L14RNA1 loaded
B1L50RNA1 loaded
B2L14RNA1+2 loaded
B2L50RNA1+4 loaded
B2L100RNA1+3 loaded
C20B2L100RNA1+3 loaded
0 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 1001, evals: 83529, top: 0.7442192493420166
round: 2, cost: 2001, evals: 155991, top: 0.7881584030720123
round: 3, cost: 3001, evals: 212496, top: 0.7881584030720123
round: 4, cost: 4001, evals: 252217, top: 0.7881584030720123
round: 5, cost: 5001, evals: 273515, top: 0.7909045576217604
round: 6, cost: 6001, evals: 294390, top: 0.7936507645510489
round: 7, cost: 7001, evals: 314447, top: 0.7936507645510489
round: 8, cost: 8001, evals: 334845, top: 0.7936507645510489
round: 9, cost: 9001, evals: 355046, top: 0.8787828650817517
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 1001, evals: 85727, top: 0.6563409418820252
round: 2, cost: 2001, evals: 16

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.5437468768175177
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 1001, evals: 75422.0, top: 0.867798089744138
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 1995, evals: 137123.0, top: 0.867798089744138
[0.37259768309025293, 0.30413365700871187, 0.3232686599010352]
round: 3, cost: 2909, evals: 184149.0, top: 0.867798089744138
[0.36504130460120765, 0.2868809164600115, 0.34807777893878084]
round: 4, cost: 3704, evals: 217490.0, top: 0.867798089744138
[0.36909541704729165, 0.27998161370127705, 0.35092296925143124]
round: 5, cost: 4441, evals: 239306.0, top: 0.867798089744138
[0.31186535867971227, 0.34103694065726253, 0.34709770066302525]
round: 6, cost: 5117, evals: 256580.0, top: 0.867798089744138
[0.33127340218509965, 0.316865274333207, 0.3518613234816933]
round: 7, cost: 5713, evals: 273283.0, top: 0.867798089744138
[0.29416537760

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 3, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 4, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 5, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 6, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 7, cost: 1, evals: 0.0, top: 0.6508485804029885
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 8, c

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 3, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 4, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 5, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 6, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 7, cost: 1, evals: 0.0, top: 0.28011203300685433
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
rou

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.6443675592872286
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 1, evals: 0.0, top: 0.6443675592872286
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 1, evals: 0.0, top: 0.6443675592872286
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 3, cost: 1, evals: 0.0, top: 0.6443675592872286
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 4, cost: 1, evals: 0.0, top: 0.6443675592872286
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 5, cost: 1, evals: 0.0, top: 0.6443675592872286
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 6, cost: 1, evals: 0.0, top: 0.6443675592872286


## BO Explorer

In [ ]:
%autoreload 2
from explorers.bo_explorer import BO_Explorer

num_of_rounds = 5
arch1 = NLNN(len(wt), hidden_dims=10, epochs=20)
arch2 = NLNN(len(wt), hidden_dims=20, epochs=20)
arch3 = NLNN(len(wt), hidden_dims=30, epochs=20)
arch4 = NLNN(len(wt), hidden_dims=40, epochs=20)
# ensemble we will use for BO Explorer
nn_model_1 = NN_model(landscape_oracle, arch1)
nn_model_2 = NN_model(landscape_oracle, arch2)
nn_model_3 = NN_model(landscape_oracle, arch3)
nn_model_4 = NN_model(landscape_oracle, arch4)
models = [nn_model_1, nn_model_2, nn_model_3, nn_model_4]
ensemble_model = Ensemble_models(models)
ensemble_model.update_model(training_sequences)

In [ ]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer()
bo_explorer.debug = True
bo_explorer.set_model(ensemble_model)
bo_explorer.run(num_of_rounds, verbose=True)

In [ ]:
from explorers.bo_explorer import BO_Explorer

bo_explorer_prod = BO_Explorer()
bo_explorer_prod.debug = False

In [ ]:
# run all evaluations 
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=5)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=5)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.scalability, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.scalability, num_starts=5)

In [ ]:
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.consistency_robustness_independence, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.consistency_robustness_independence, num_starts=5)